# Multi search agent Rag 

* Arxiv  Wikipedia  pdf 

This project implements a multi-search agent using Retrieval-Augmented Generation (RAG) to query and retrieve information from multiple sources, including Wikipedia, PDF documents, and ArXiv. The system also includes a chatbot interface for user interaction.

In [38]:
import numpy as np
import pandas as pd 
import os
from dotenv import load_dotenv
load_dotenv()
API_Hugging = os.getenv('HUGGING_API')

Gorq_Api = os.getenv('GORQ_API')

In [39]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

In [40]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [41]:
wikipedia_Wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
Wiki_tool = WikipediaQueryRun(api_wrapper=wikipedia_Wrapper)

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

In [42]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(api_key=API_Hugging,model_name="sentence-transformers/all-MiniLM-l6-v2")
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)


In [43]:
retriever=vectordb.as_retriever()

retriever function in LangChain is typically used to fetch relevant information or documents from a dataset, database, or vector store based on a user query

In [54]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,
                                     name="information about deep learing",
                                     description='Search informations about Machine learing and deep learing.you must use this tool')

In [55]:
# Arxiv Tool
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wapper = ArxivAPIWrapper(
    top_k_results = 2,
    ARXIV_MAX_QUERY_LENGTH = 300,
    load_max_docs = 3,
    load_all_available_meta = False,
    doc_content_chars_max = 200
)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wapper)
arxiv.name

'arxiv'

In [56]:
# combain 
tools =[arxiv,Wiki_tool,retriever_tool]

In [57]:
# llm model 
from langchain_groq import ChatGroq
chat = ChatGroq(temperature=0.1, groq_api_key=Gorq_Api, model_name="deepseek-r1-distill-llama-70b")


In [58]:
# agent 
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain.agents import create_openai_tools_agent

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant"),
  ("placeholder", "{chat_history}"),
  ("human", "{input}"),
  ("placeholder", "{agent_scratchpad}"),
]
)

agent= create_openai_tools_agent(llm=chat,tools=tools,prompt=prompt)



In [59]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)



In [60]:
agent_executor.invoke({"input":"tell me about machine learning"})



> Entering new AgentExecutor chain...

Invoking: `information about deep learing` with `{'query': 'machine learning'}`


Get started by adding tracing to your application.
Create dashboards to view key metrics like RPS, error rates and costs.

Evals​
The quality and development speed of AI applications depends on high-quality evaluation datasets and metrics to test and optimize your applications on. The LangSmith SDK and UI make building and running high-quality evaluations easy.

Get started by creating your first evaluation.
Quickly assess the performance of your application using our off-the-shelf evaluators (Python only) as a starting point.
Analyze results of evaluations in the LangSmith UI and compare results over time.
Easily collect human feedback on your data to improve your application.

Prompt Engineering​
While traditional software applications are built by writing code, AI applications involve writing prompts to instruct the LLM on what to do. LangSmith provides a set of

{'input': 'tell me about machine learning',
 'output': 'Machine learning (ML) is a field of study within artificial intelligence (AI) that focuses on the development and use of algorithms that can learn from and make decisions based on data. These algorithms enable machines to improve their performance on specific tasks without being explicitly programmed for those tasks. Machine learning is widely used in various applications, including image and speech recognition, natural language processing, predictive analytics, and more.'}